In [1]:
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup 

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## Data collection

In [2]:
blr_data=pd.read_csv(r'D:\Academics\blr_neighborhoods.csv')

In [3]:
blr_data.head()

,Neighborhood,Latitude,Longitude
0,Agram,45.813177,15.977048
1,Amruthahalli,13.066513,77.596624
2,Attur,11.663711,78.533551
3,Banaswadi,13.014162,77.651854
4,Bellandur,58.235358,26.683117


In [4]:
blr_data.shape[0]

352

## Bangalore Geodata

In [5]:
address = 'Bangalore, Karnataka'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 12.9791198, 77.5912997.


In [6]:
# create map of New York using latitude and longitude values
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng,neighborhood in zip(blr_data['Latitude'], blr_data['Longitude'],blr_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

## Let us explore first neighborhood in the dataset

In [7]:
CLIENT_ID = 'FJRUV405IVRGSD3O2B4HGDHXQ0I204N0CIFHYTOTFJ3GQK5L' # your Foursquare ID
CLIENT_SECRET = 'ZF5GRN3JDNBNBFKJ5DZLPFOC3Q3IYXYU3EKXF2P5VVKJS3QC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FJRUV405IVRGSD3O2B4HGDHXQ0I204N0CIFHYTOTFJ3GQK5L
CLIENT_SECRET:ZF5GRN3JDNBNBFKJ5DZLPFOC3Q3IYXYU3EKXF2P5VVKJS3QC


In [8]:
blr_data.loc[0,'Neighborhood']

'Agram'

In [9]:
neighborhood_latitude = blr_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = blr_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = blr_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Agram are 45.813177, 15.977048.


## Making API call to know near by venues of first neighborhood

In [10]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={} &client_secret={} &v={} &ll={},{} &radius=500 &limit=100'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?&client_id=FJRUV405IVRGSD3O2B4HGDHXQ0I204N0CIFHYTOTFJ3GQK5L &client_secret=ZF5GRN3JDNBNBFKJ5DZLPFOC3Q3IYXYU3EKXF2P5VVKJS3QC &v=20180605 &ll=45.813177,15.977048 &radius=500 &limit=100'

In [11]:
results = requests.get(url).json()


In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-13-4243e3e1ab9c>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Amélie,Dessert Shop,45.813842,15.979011
1,Vinodol,Mediterranean Restaurant,45.811666,15.975643
2,Escape Room Enigmarium Zagreb,Escape Room,45.815129,15.976299
3,A Burger & Champagne Bar,Restaurant,45.813387,15.980151
4,Gajbica,Bistro,45.813531,15.979550
5,Capuciner Grill & Steak,Steakhouse,45.815103,15.978144
6,Main Square Hostel,Hostel,45.814026,15.976418
7,Amfora Bistro,Mediterranean Restaurant,45.814261,15.977238
8,Express Bar,Coffee Shop,45.811987,15.979285
9,Corner bar,Bar,45.812930,15.979440


In [14]:
print('{} venues were returned by Foursquare for one(1st) neighborhood.'.format(nearby_venues.shape[0]))

86 venues were returned by Foursquare for one(1st) neighborhood.


In [15]:
# create map of Manhattan using latitude and longitude values
map_first_neigh = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_first_neigh)  
    
map_first_neigh

## So far we have been doing for only one neighborhood
## Now let us do this for all the neighborhoods

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
neigh_venues = getNearbyVenues(names=blr_data['Neighborhood'],
                                   latitudes=blr_data['Latitude'],
                                   longitudes=blr_data['Longitude']
                                  )

In [18]:
neigh_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agram,45.813177,15.977048,Amélie,45.813842,15.979011,Dessert Shop
1,Agram,45.813177,15.977048,Vinodol,45.811666,15.975643,Mediterranean Restaurant
2,Agram,45.813177,15.977048,Escape Room Enigmarium Zagreb,45.815129,15.976299,Escape Room
3,Agram,45.813177,15.977048,A Burger & Champagne Bar,45.813387,15.980151,Restaurant
4,Agram,45.813177,15.977048,Gajbica,45.813531,15.979550,Bistro


In [19]:
neigh_venues.shape

(726, 7)

### Let's find how many venues were calculated for each neighborhood

In [76]:
df=neigh_venues.groupby('Neighborhood').count()
df

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Achitnagar,2,2,2,2,2,2
Adugodi,5,5,5,5,5,5
Agram,86,86,86,86,86,86
Akkur,26,26,26,26,26,26
Alahalli,1,1,1,1,1,1
Amruthahalli,4,4,4,4,4,4
Anekal,2,2,2,2,2,2
Arudi,3,3,3,3,3,3
Attibele,4,4,4,4,4,4


In [78]:
print('There are {} uniques categories.'.format(len(neigh_venues['Venue Category'].unique())))

There are 168 uniques categories.


## One hot encoding for catogorizing the values

In [79]:
# one hot encoding
blr_onehot = pd.get_dummies(neigh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blr_onehot['Neighborhood'] = neigh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [blr_onehot.columns[-1]] + list(blr_onehot.columns[:-1])
blr_onehot = blr_onehot[fixed_columns]

blr_onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Arcade,Art Museum,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Baseball Field,Beach,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Campground,Candy Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Doner Restaurant,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Other Nightlife,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Rajasthani Restaurant,Resort,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Travel & Transport,Tunnel,Turkish Coffeehouse,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Agram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Agram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Agram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Agram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Agram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [80]:
blr_onehot.shape

(726, 169)

In [81]:
blr_grouped = blr_onehot.groupby('Neighborhood').mean().reset_index()
blr_grouped.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Arcade,Art Museum,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Baseball Field,Beach,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Campground,Candy Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Doner Restaurant,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Other Nightlife,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Rajasthani Restaurant,Resort,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Travel & Transport,Tunnel,Turkish Coffeehouse,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Achitnagar,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Adugodi,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.2,0.0,0.0,0.200000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000

## Knowing the top 5  venues in each neighborhood

In [82]:
num_top_venues = 5

for hood in blr_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =  blr_grouped[ blr_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Achitnagar----
             venue  freq
0       Food Truck   0.5
1     Burger Joint   0.5
2            Plaza   0.0
3      Music Venue   0.0
4  Motorcycle Shop   0.0


----Adugodi----
                venue  freq
0         Bus Station   0.2
1      Ice Cream Shop   0.2
2          Playground   0.2
3                Café   0.2
4  Athletics & Sports   0.2


----Agram----
           venue  freq
0            Bar  0.08
1  Historic Site  0.07
2           Café  0.07
3         Bistro  0.06
4          Plaza  0.05


----Akkur----
              venue  freq
0       Supermarket  0.23
1               Bar  0.08
2  Department Store  0.08
3          Pharmacy  0.08
4         Pet Store  0.08


----Alahalli----
               venue  freq
0  Food & Drink Shop   1.0
1    Other Nightlife   0.0
2    Motorcycle Shop   0.0
3           Mountain   0.0
4      Movie Theater   0.0


----Amruthahalli----
               venue  freq
0  Indian Restaurant  0.25
1  Convenience Store  0.25
2         Shoe Store  0.25
3      

### Returning most common venues (order wise) in each neighborhood

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [84]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = blr_grouped['Neighborhood']

for ind in np.arange(blr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(blr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Achitnagar,Food Truck,Burger Joint,Plaza,Music Venue,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex
1,Adugodi,Bus Station,Ice Cream Shop,Playground,Café,Athletics & Sports,ATM,Other Nightlife,Movie Theater,Moving Target,Multicuisine Indian Restaurant
2,Agram,Bar,Historic Site,Café,Bistro,Plaza,Mediterranean Restaurant,Dessert Shop,Restaurant,Hostel,Hotel
3,Akkur,Supermarket,Bar,Department Store,Pharmacy,Pet Store,Clothing Store,Shopping Mall,Chinese Restaurant,Furniture / Home Store,Toy / Game Store
4,Alahalli,Food & Drink Shop,Other Nightlife,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue
5,Amruthahalli,Indian Restaurant,Convenience Store,Shoe Store,Print Shop,Playground,Pizza Place,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant
6,Anekal,ATM,Camera Store,Other Nightlife,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue
7,Arudi,Gym,Hotel,Museum,Other Nightlife,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex
8,Attibele,ATM,Bakery,South Indian Restaurant,Indian Restaurant,Arcade,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex
9,Bagalgunte,ATM,Bakery,Pizza Place,Department Store,Indian Restaurant,Art Museum,Pub,Movie Theater,Moving Target,Multicuisine Indian Restaurant


In [85]:
kclusters = 5

blr_grouped_clustering = blr_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(blr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [86]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [101]:
neighborhoods_venues_sorted.tail()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,1,Virgonagar,ATM,Breakfast Spot,Italian Restaurant,Jewelry Store,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
98,1,Whitefield,Gun Shop,Pizza Place,Ice Cream Shop,Convenience Store,Donut Shop,Sandwich Place,Baseball Field,Grocery Store,ATM,Moving Target
99,0,Yelachenahalli,Indian Restaurant,Pizza Place,Metro Station,Motorcycle Shop,Playground,Museum,Mountain,Movie Theater,Plaza,Moving Target
100,1,Yelahanka,ATM,Pizza Place,Jewelry Store,Train Station,Playground,Pie Shop,Motorcycle Shop,Mountain,Movie Theater,Moving Target
101,1,Yeliyur,Train Station,ATM,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue


In [102]:
blr_clus = blr_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
blr_clus =blr_clus.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

blr_clus.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,45.813177,15.977048,1.0,Bar,Historic Site,Café,Bistro,Plaza,Mediterranean Restaurant,Dessert Shop,Restaurant,Hostel,Hotel
1,Amruthahalli,13.066513,77.596624,0.0,Indian Restaurant,Convenience Store,Shoe Store,Print Shop,Playground,Pizza Place,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant
2,Attur,11.663711,78.533551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Banaswadi,13.014162,77.651854,0.0,Vegetarian / Vegan Restaurant,Indian Restaurant,BBQ Joint,Park,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
4,Bellandur,58.235358,26.683117,3.0,Restaurant,ATM,Other Nightlife,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum


## As few neighbourhood are very remote, the venues could not be calculated so those neighborhoods have been neglected

In [104]:
blr_clus.shape

(352, 14)

In [135]:
blr_new=blr_clus.dropna().reset_index(drop=True)

In [136]:
blr_new.shape

(106, 14)

In [137]:
blr_new['Cluster Labels']=blr_new['Cluster Labels'].astype(int)

In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blr_new['Latitude'], blr_new['Longitude'], blr_new['Neighborhood'], blr_new['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [138]:
blr_new.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,45.813177,15.977048,1,Bar,Historic Site,Café,Bistro,Plaza,Mediterranean Restaurant,Dessert Shop,Restaurant,Hostel,Hotel
1,Amruthahalli,13.066513,77.596624,0,Indian Restaurant,Convenience Store,Shoe Store,Print Shop,Playground,Pizza Place,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant
2,Banaswadi,13.014162,77.651854,0,Vegetarian / Vegan Restaurant,Indian Restaurant,BBQ Joint,Park,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
3,Bellandur,58.235358,26.683117,3,Restaurant,ATM,Other Nightlife,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
4,Bhattarahalli,13.025800,77.714279,1,Ice Cream Shop,Juice Bar,Hotel,Udupi Restaurant,Café,ATM,Other Nightlife,Movie Theater,Moving Target,Multicuisine Indian Restaurant


In [140]:
blr_new.loc[blr_new['Cluster Labels'] == 0, blr_new.columns[[0]+ list(range(3,blr_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Amruthahalli,0,Indian Restaurant,Convenience Store,Shoe Store,Print Shop,Playground,Pizza Place,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant
2,Banaswadi,0,Vegetarian / Vegan Restaurant,Indian Restaurant,BBQ Joint,Park,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
5,Byatarayanapura,0,Indian Restaurant,Department Store,Andhra Restaurant,Wine Shop,Kerala Restaurant,Flower Shop,Gourmet Shop,Other Nightlife,Moving Target,Multicuisine Indian Restaurant
6,Doddanekkundi,0,Indian Restaurant,Mediterranean Restaurant,Intersection,Restaurant,Coffee Shop,Multiplex,Fast Food Restaurant,Sports Bar,Movie Theater,Fried Chicken Joint
7,Domlur,0,Indian Restaurant,Café,Sports Bar,Liquor Store,Lounge,Food & Drink Shop,Pizza Place,Pub,Rajasthani Restaurant,Italian Restaurant
8,EPIP,0,Indian Restaurant,Hotel,Cafeteria,Breakfast Spot,Mediterranean Restaurant,Maharashtrian Restaurant,Fried Chicken Joint,Korean Restaurant,Movie Theater,Moving Target
11,Hoodi,0,Indian Restaurant,Breakfast Spot,Yoga Studio,Andhra Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
12,Horamavu,0,Indian Restaurant,Chinese Restaurant,Pharmacy,Athletics & Sports,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
15,Kundalahalli,0,Indian Restaurant,Hotel,Lounge,Coffee Shop,Bakery,Pizza Place,Pie Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant
22,Sadashivanagar,0,Indian Restaurant,Department Store,Coffee Shop,Café,Women's Store,Seafood Restaurant,Plaza,Performing Arts Venue,Yoga Studio,BBQ Joint


In [141]:
blr_new.loc[blr_new['Cluster Labels'] == 1, blr_new.columns[[0]+ list(range(3,blr_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,1,Bar,Historic Site,Café,Bistro,Plaza,Mediterranean Restaurant,Dessert Shop,Restaurant,Hostel,Hotel
4,Bhattarahalli,1,Ice Cream Shop,Juice Bar,Hotel,Udupi Restaurant,Café,ATM,Other Nightlife,Movie Theater,Moving Target,Multicuisine Indian Restaurant
9,Fraser Town,1,Food,Market,Other Nightlife,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue
10,HighCourt,1,Cosmetics Shop,Park,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Other Nightlife
13,Indiranagar S.O (Bangalore),1,Pub,Restaurant,Café,Steakhouse,Indian Restaurant,Lounge,Hotel,Gym,Grocery Store,Music Venue
14,Kadugodi,1,Bakery,Indian Sweet Shop,Multicuisine Indian Restaurant,Coffee Shop,Pedestrian Plaza,Movie Theater,Moving Target,Multiplex,Museum,Music Venue
16,Lingarajapuram,1,Health & Beauty Service,Bakery,Café,Burger Joint,Andhra Restaurant,Plaza,Other Nightlife,Movie Theater,Moving Target,Pool
17,Medimallasandra,1,Scenic Lookout,ATM,Other Nightlife,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
18,Mundur,1,Bakery,ATM,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Other Nightlife
19,NAL,1,Trail,Pizza Place,Cafeteria,ATM,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum


In [142]:
blr_new.loc[blr_new['Cluster Labels'] == 2, blr_new.columns[[0]+ list(range(3,blr_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,Budigere,2,Snack Place,ATM,Music Venue,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
96,Nelamangala,2,Snack Place,Clothing Store,ATM,Park,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue
100,Sarjapura,2,Snack Place,ATM,Music Venue,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum


In [143]:
blr_new.loc[blr_new['Cluster Labels'] == 3, blr_new.columns[[0]+ list(range(3,blr_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Bellandur,3,Restaurant,ATM,Other Nightlife,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum
47,Shanthinagar,3,Restaurant,ATM,Other Nightlife,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum


In [144]:
blr_new.loc[blr_new['Cluster Labels'] == 4, blr_new.columns[[0]+ list(range(3,blr_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Sakalavara,4,Farm,ATM,Other Nightlife,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue
81,Hosadurga,4,Farm,ATM,Other Nightlife,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue
